# Robustness and Plasticity in Regulatory Networks

* [Introduction](./RPRN-Introduction.ipynb)
* [BoolNet](#BoolNet)
    * [Network construction](#Network-construction)
    * [Attractors](#Attractors)
    * [Labeling](#Labeling)
* [Functions](./RPRN-Functions.ipynb)
* [Updating](./RPRN-Updating.ipynb)
* [States](./RPRN-States.ipynb)
* [Appendix](./RPRN-Appendix.ipynb)

# BoolNet

Once a regulatory network has been constructed there exist [multiple tools](http://www.colomoto.org/software/) to analyse it. In this tutorial we will focus in the R package [Boolnet](https://cran.r-project.org/web/packages/BoolNet/index.html).

It is possible to combine the different software tools to make a better analysis, the standar format for saving a logical regulatory network is [SBML-qual](http://www.colomoto.org/formats/sbml-qual.html) (see the [Appendix](./RPRN-Appendix.ipynb) for more information).

Sometimes it is useful to [simplify the network](http://ginsim.org/ginsim-doc/current/algo-reduction.html). 

## Network construction

The interactions of a biological system can be expressed as a set of logical functions using multiple formalisms ([Boolean functions](https://en.wikipedia.org/wiki/Boolean_function), [truth tables](https://en.wikipedia.org/wiki/Truth_table), [Binary Decision Diagrams](https://en.wikipedia.org/wiki/Binary_decision_diagram)). BoolNet accepts a set of boolean functions.

For examplet the Th17/iTreg network can be expressed as a set of boolean functions obtained from the known interactions among the cytokines and transcription factors.

>targets, functions

>IL2, (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))

>RORGT, (STAT3 & TGFB) &  ! FOXP3

>STAT3, (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)

>FOXP3, (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)

>TGFB, TGFBe | ((TGFB | FOXP3) &  ! STAT3 )

>IL10, IL10e | (IL10 & (STAT3 | TGFB))

First, we will load BoolNet

In [ ]:
#Uncomment next line if you haven't installed BoolNet
#install.packages("BoolNet", repos='http://cran.us.r-project.org')
library(BoolNet)
source("BoolNet-extensions.R")

Boolnet creates a network object with the function __loadNetwork()__. This function accepts the filename of the file with the rules. In this case we will create the file from the notebook.

In [ ]:
fileConn<-file("minTh17iTreg.txt")
writeLines(c(
    "targets, functions",
    "IL2, (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))",
    "RORGT, (STAT3 & TGFB) &  ! FOXP3",
    "STAT3, (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)",
    "FOXP3, (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)",
    "TGFB, TGFBe | ((TGFB | FOXP3) &  ! STAT3 )",
    "IL10, IL10e | (IL10 & (STAT3 | TGFB))", 
    "IL2e, IL2e",
    "IL21e, IL21e",
    "TGFBe, TGFBe",
    "IL10e, IL10e"
    ), fileConn)
close(fileConn)

Now we can create a network object. If you don't specify a node BoolNet will raise a warning and asumme it is an input. The resulting network should have ten nodes.

In [ ]:
net <- loadNetwork("minTh17iTreg.txt")
net

The result is a network object. This object structure is:
* __network__
    * __\$interactions__ : list of nodes where each node has its functions represented as truth tables. (Use iterator or [[n]] to examine each node)
        * __\$input__ : index of regulators
        * __\$func__ : function as truth table
        * __\$expression__ : Boolean function as string
    * __\$genes__ : ordered node names
    * __\$fixed__ : which nodes have fixed values (see [Functions](./RPRN-Functions.ipynb))

In [ ]:
net$interactions[[1]]
net$genes
net$fixed

## Attractors

[//]: # (As the state of the network is updated using the functions, the network will reach a previously visited state called an attractor. The set of states that lead to an attractor is called the basin of the attractor [Fig1 C] \cite{Gerherson2004}.)

Attractors represent stable states in the dynamics of the network and have been related to cell types or  biological processes like the cell cycle. 

The number of possible states and the complexity of finding the attractors grows exponentially with the number of nodes. It is possible efficiently find the stable states [without simulations](http://ginsim.org/documentation) and using [model checking](http://dl.acm.org/citation.cfm?id=2014689). 

The attractors we obtain can be affected by the [updating method](./RPRN-Updating.ipynb) (synchronus or asynchronous).

BoolNet uses the function __getAttractors()__ to obtain the attractors of the network. This function has a lot of options that can be checked in the documentation.

In [ ]:
?getAttractors

Here, we will use the default options

In [ ]:
attr <- getAttractors(net)
attr

The result is an attractor object. States are saved as an integer. This object structure can change depending of the options used. The general structure is:
* __attractor__
    * __\$stateInfo__ : information of the simulation, evaluated states and transition table
        * __\$table__,  __\$attractorAssignment__,  __\$stepsToAttractor__,  __\$initialStates__,  __\$genes__,  __\$fixedGenes__,  
    * __\attractors$__ : list of attractors. (Use iterator or [[n]] to examine each node)
        * __\$involvedStates__ : states that make the attractor
        * __\$basinSize__ : states that reach the attractor

In [ ]:
attr$attractors[[58]]

We can create a dataframe including only the state and the basin size. However, there are some attractors that have more than one state, which can complicate the process. To solve this we will use the function __attractor2dataframe()__ that transforms the attractors to dataframes. As we can have cyclic states this function will convert the to characters and join them with "\".

In [ ]:
attr.df <- attractor2dataframe(attr) 
attr.df

It would also be useful to have the attractors in 0s and 1s format and know the gene names. Lets make a function for this

In [ ]:
dec2binState <- function(x, genes){ 
    state <- as.integer( intToBits(x)[1:length(genes)] )  
    names(state) <- genes
    state
    }

Using this function we can analyze the active nodes in a state. For example, the attractor 58:

In [ ]:
bin.states <- lapply (attr$attractors[[58]]$involvedStates, 
                      dec2binState, net$genes)
bin.states

## Labeling

Attractors correspond to cell types. It is very important to verify that all the expected cell types appear in our attractors, if they are not present we might be missing interactions. It is also important to see if there are attractors that do not correspond to known cell types, as they may be predictions or show errors in the construction of the network.

However, when there are a lot of inputs, it is possible that many attractors correspond to a single cell type that can be found in different environments (we will discuss this further). To solve this problem we will label our attractors according to a set of rules.

In the case of the Th17/iTreg network this rules are:

>labels,  rules

>Th0,   ! (RORGT | FOXP3 | TGFB | IL10)

>Th17,  RORgt & STAT3

>iTreg, FOXP3 & TGFB

>IL10+, IL10 & ! (RORGT | FOXP3 | TGFB)

>TGFB+, TGFB & ! (RORGT | FOXP3 | TGFB)

>IL10+TGFB+, (TGFB & IL10) & ! (RORGT | FOXP3)

In [ ]:
labels.rules <- data.frame(
    labels = c('Th0', 'Th17', 'Treg', 'IL10+', 'TGFB+'),
    rules  = c('!(RORGT | FOXP3 | TGFB | IL10)', 
            'RORGT & STAT3', 
            'FOXP3 & TGFB', 
            'IL10', 
            'TGFB & ! (RORGT | FOXP3)' ),
    stringsAsFactors = FALSE
)
labels.rules

For each rule we can write a function that evaluates  the nodes. For example, for the rule __Th0__ we could write the following rule:

In [ ]:
ruleTh0 <- function(
    IL2, RORGT, STAT3, FOXP3, TGFB, IL10, IL2e, IL21e, TGFBe, IL10e
    ) { 
    if (!(RORGT | FOXP3 | TGFB | IL10)) 'Th0' 
}

We can use this function with a binary state. For examplem the state 0 (0,0,0,0,0,0,0,0,0,0) has the label "Th0", but the state 121 (1,0,0,1,1,1,1,,0,0,0) does not.

In [ ]:
label <- ruleTh0(0,0,0,0,0,0,0,0,0,0)
label
state <- ruleTh0(1,0,0,1,1,1,1,0,0,0)
state

We can also give the function a list of named arguments using __do.call()__, this way the function can receive the state obtained from __dec2binState()__.

In [ ]:
state <- attr$attractors[[1]]$involvedStates
state <- dec2binState(state, net$genes)
#state
label <- do.call(ruleTh0, as.list(state))
label

While we could create specific functions for each labeling rule, it is better to make a general function that can interpret our rules. 
The function __labelState()__ takes a binary state with gene names, a set of labels and the rules as strings and returns a label.

In [ ]:
state <- dec2binState(121, net$genes)
#state
label <- labelState(state, net$genes, labels.rules$labels, labels.rules$rules)
label

Using this method it is possible to label all the states involved in an attractor. The function __labelAttractors()__ takes an attractors object created by BoolNet and returns a list of the labels for each attractor in order. If an attractor has multiple states it will return a label for each state.

Lets obtain the labels of all the attractors using the function __labelAttractors()__.

In [ ]:
labels <- labelAttractors(attr, net$genes, labels.rules$labels, labels.rules$rules)
labels

We will add the labels to the dataframe we created with __attractor2dataframe()__. To do this we will need to join the labels of the attractors that have more than one state with "/".

In [ ]:
attr.df$label <- sapply(labels, function(label) {
    paste(as.character(label), collapse='/')
})
attr.df


As we can see some labels have more than one attractor. This is because some cell types can be found in different environments. We can see all the unique labels to determine if all the labels correspond to the expected cell types.

In [ ]:
unique(attr.df$label)

While we have all the expected cell types we have some cycles where one of the states could not be labeled. If we check the dataframe, the states 338 and 466 have no label, we can study them to see what happened.

In [ ]:
state.odd.a1 <- dec2binState(338, net$genes)
state.odd.a2 <- dec2binState(341, net$genes)
state.odd.a1
state.odd.a2
state.odd.b1 <- dec2binState(466, net$genes)
state.odd.b2 <- dec2binState(469, net$genes)
state.odd.b1
state.odd.b2

As we can see this states correspond to oscillations where ROR$\gamma$t can not stabilish a stable expression. If we use an asynchronous update this states will dissapear. 

We can create a new rule to include this states.
> RORGT+, RORGT & ! STAT3

In [ ]:
labels.rules <- rbind(labels.rules, c('RORGT+', 'RORGT & ! STAT3'))
labels.rules

We will need to do the labeling and the dataframe once more.

In [ ]:
labels <- labelAttractors(attr, net$genes, labels.rules$labels, labels.rules$rules)
labels <- sapply(labels, function(l) paste(as.character(l), collapse='/'))
attr.df$label <- labels
attr.df


Now that all the states are labeled we can also determine which attractor states correspond to each label and the joint size of the basin of attraction of each label.

In [ ]:
attractors.by.label <- data.frame(
    states = tapply(attr.df$involvedStates, attr.df$label, paste),
    basin = tapply(attr.df$basin, attr.df$label, sum)
)
attractors.by.label

This ends the basic BoolNet tutorial.

# Next

* [Introduction](./RPRN-Introduction.ipynb)
* BoolNet
* [Functions](./RPRN-Functions.ipynb)
* [Updating](./RPRN-Updating.ipynb)
* [States](./RPRN-States.ipynb)
* [Appendix](./RPRN-Appendix.ipynb)